In [10]:
import pandas as pd
df = pd.read_csv("datascience.csv", encoding="gb18030")
df.head()

,key
0,不过，你们或许会问，我们请你来谈谈妇女与小说——可是，这与自己的房间又有什么关系呢？我会与大...
1,现在，要是可以请你们继续听我说的话，场景已经换了地方。落叶依旧纷纷，却已是伦敦，不再是牛桥了...
2,傍晚已至，却连一句有分量的说法、一个确凿无疑的事实都没能带回来，未免让人失望。妇女之所以比男...
3,我们发现，在16世纪，显然找不到一位心境如此的妇女。只要想一想伊丽莎白时代雕刻在墓碑上的那些...
4,终于，我在一番信步闲逛之后，还是来到了放着在世作家作品的书架前。既有女人的，也有男人的，因为...


In [71]:
import jieba
import jieba.analyse

In [73]:
def chinese_word_cut(mytext):
    tag=jieba.analyse.extract_tags(mytext,topK=100,withWeight=True,allowPOS=('nr'))
    return " ".join(i[0] for i in tag)

In [74]:
df["content_cutted"] = df.key.apply(chinese_word_cut)

In [75]:
df.content_cutted.head()

0    我们 什么 学院 你们 小说 午餐会 妇女 自己 心房 那些 幻象 学监 有人 这个 教堂 ...
1    妇女 男人 女人 他们 自己 她们 教授 真理 你们 这样 这些 那些 钞票 这儿 那位 我...
2    妇女 女人 男人 自己 我们 她们 先生 事实 天才 他们 这些 某位 历史 书架 任何 时...
3    她们 小说 女人 妇女 我们 自己 小说家 男人 句子 这样 人们 那些 诗歌 情感 他们 ...
4    女人 她们 男人 小说 我们 自己 这样 性别 什么 妇女 句子 房间 那些 这些 创造力 ...
Name: content_cutted, dtype: object

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 1.0,
                                min_df = 5)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [42]:
from sklearn.decomposition import LatentDirichletAllocation

In [83]:
n_topics=10
lda= LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [84]:
lda.fit(tf)

D:\Anaconda\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [85]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print() 
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
妇女 之中 一切 小说家 先生 性别 教授 有人 他们 她们
Topic #1:
这个 那里 小说家 她们 男人 这种 先生 我们 这样 女人
Topic #2:
房间 一样 她们 女人 诗人 有人 这些 什么 你们 那些
Topic #3:
房间 什么 他们 小说 你们 这种 那些 先生 事实 男人
Topic #4:
这么 房间 妇女 有人 自己 一切 一样 什么 人们 任何
Topic #5:
人们 房间 妇女 那样 小说 一样 那些 我们 那里 你们
Topic #6:
这么 这样 女性 这儿 性别 这种 小说家 男人 这个 这些
Topic #7:
我们 这样 自己 一样 任何 这些 这个 这种 女人 那些
Topic #8:
一切 她们 你们 那样 教授 这儿 人们 这种 作品 房间
Topic #9:
这种 妇女 作品 那样 这样 他们 我们 性别 小说家 诗人



In [86]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.002339  0.000010       1        1  97.188047
4      0.000111  0.000384       2        1   0.312439
1      0.000002 -0.000285       3        1   0.312439
6     -0.000019 -0.000348       4        1   0.312439
3      0.000180  0.000126       5        1   0.312439
0      0.000620 -0.000280       6        1   0.312439
8      0.001103  0.000101       7        1   0.312439
2      0.000043  0.000216       8        1   0.312439
9      0.000178 -0.000061       9        1   0.312439
5      0.000120  0.000137      10        1   0.312439, topic_info=     Category      Freq Term     Total  loglift  logprob
term                                                    
13    Default  6.000000   她们  6.000000  30.0000  30.0000
30    Default  6.000000   这种  6.000000  29.0000  29.0000
5     Default  6.000000   什么  6.000000  28.0000  28.0000
1     Default  6.000000   一样  6.000000  27.0000  27.0000
14    Default  6.000000   妇女  6.000000  26.0000  26.0000
29    Default  6.000000   这样  6.000000  25.0000  25.0000
31    Default  6.000000   那么  6.000000  24.0000  24.0000
32    Default  6.000000   那些  6.000000  23.0000  23.0000
6     Default  6.000000   他们  6.000000  22.0000  22.0000
27    Default  6.000000   这些  6.000000  21.0000  21.0000
25    Default  6.000000   这个  6.000000  20.0000  20.0000
11    Default  6.000000   女人  6.000000  19.0000  19.0000
18    Default  6.000000   我们  6.000000  18.0000  18.0000
23    Default  6.000000   自己  6.000000  17.0000  17.0000
22    Default  6.000000   男人  6.000000  16.0000  16.0000
7     Default  6.000000   任何  6.000000  15.0000  15.0000
15    Default  6.000000   小说  6.000000  14.0000  14.0000
4     Default  6.000000   人们  6.000000  13.0000  13.0000
21    Default  6.000000   有人  6.000000  12.0000  12.0000
3     Default  6.000000   事实  6.000000  11.0000  11.0000
10    Default  6.000000   先生  6.000000  10.0000  10.0000
28    Default  5.000000   这儿  5.000000   9.0000   9.0000
24    Default  6.000000   诗人  6.000000   8.0000   8.0000
34    Default  6.000000   那里  6.000000   7.0000   7.0000
12    Default  6.000000   女性  6.000000   6.0000   6.0000
19    Default  5.000000   房间  5.000000   5.0000   5.0000
0     Default  5.000000   一切  5.000000   4.0000   4.0000
33    Default  5.000000   那样  5.000000   3.0000   3.0000
26    Default  5.000000   这么  5.000000   2.0000   2.0000
16    Default  5.000000  小说家  5.000000   1.0000   1.0000
...       ...       ...  ...       ...      ...      ...
8     Topic10  0.019464   作品  5.074416   0.2051  -3.5506
2     Topic10  0.019418   之中  5.072552   0.2031  -3.5530
20    Topic10  0.019159   教授  5.071773   0.1898  -3.5664
16    Topic10  0.018537  小说家  5.059860   0.1592  -3.5994
0     Topic10  0.018402   一切  5.074371   0.1490  -3.6067
17    Topic10  0.018071   性别  5.066522   0.1324  -3.6248
4     Topic10  0.020566   人们  6.017030   0.0898  -3.4955
34    Topic10  0.019870   那里  6.021626   0.0546  -3.5299
3     Topic10  0.019441   事实  6.014361   0.0340  -3.5518
24    Topic10  0.019311   诗人  6.005861   0.0287  -3.5585
12    Topic10  0.019282   女性  6.013210   0.0260  -3.5599
21    Topic10  0.018956   有人  6.003038   0.0106  -3.5770
10    Topic10  0.018782   先生  6.011320   0.0000  -3.5862
28    Topic10  0.018328   这儿  5.989609  -0.0209  -3.6107
14    Topic10  0.020437   妇女  6.939726  -0.0592  -3.5018
15    Topic10  0.020275   小说  6.946272  -0.0681  -3.5098
1     Topic10  0.020200   一样  6.961948  -0.0740  -3.5135
32    Topic10  0.020131   那些  6.954634  -0.0764  -3.5169
18    Topic10  0.019973   我们  6.969970  -0.0865  -3.5248
6     Topic10  0.019646   他们  6.944054  -0.0993  -3.5413
7     Topic10  0.019505   任何  6.956824  -0.1083  -3.5485
25    Topic10  0.019236   这个  6.957378  -0.1222  -3.5623
5     Topic10  0.019219   什么  6.952892  -0.1225  -3.5633
23    Topic10  0.019170   自己  6.960721  -0.1262  -3.5658
31    Topic10  0.019136   那么  6.952791  -0.1268  -3.5676
30 